LS DS 413 assignment  

note: answers at the end


In [22]:

import csv
import re
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

from numpy import argmax
from sklearn.preprocessing import LabelEncoder

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to /home/dliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
df= pd.read_csv('projects/DS-Unit-4-Sprint-1-NLP/module2-vector-representations/train.csv')

In [63]:
## drop the 2s just for shiggles

cond = df['ratingCategory'] != 2
df = df[cond]

In [64]:
def clean_text(X):
    """
        customized cleaning function
        X: series
    
        return: np.array
    """
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    number = re.compile('[0-9]+\s')  #remove freestanding numbers

    X = X.map(lambda t : t.lower()) # lwercase text
    X = X.map(lambda t : re.sub("\\\\n", ' ', t))
    X = X.map(lambda t : re.sub("\\\\x..", '',t))   
    X = X.map(lambda t : re.sub("^b[\'\"]", '',t))   
    X = X.map(lambda t : BeautifulSoup(t, "lxml").text) # strip html tags
    X = X.map(lambda t : REPLACE_BY_SPACE_RE.sub(' ', t))  # symbols by space in text
    X = X.map(lambda t : BAD_SYMBOLS_RE.sub('', t)) # delete symbols which are in BAD_SYMBOLS_RE
    X = X.map(lambda t : number.sub('', t))         # remove groups of digits
    X = X.map(lambda t : ' '.join(word for word in t.split() if word not in STOPWORDS))# delete stopwords
    return np.array(X)

clean = FunctionTransformer(clean_text)

In [66]:
svcpipe = Pipeline([('clean', clean),
                  ('vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                  ('clf', SVC())
                   ])
 
parameters = {
    'vect__max_df': ( 0.7, 0.75, 0.8),
    'vect__min_df': (.01, .02, .03),
    'vect__max_features': (300,500,600),
    'clf__kernel':('linear','poly','rbf'),
    'clf__C':([.5,1])
}
data = df.description
target = df.ratingCategory
grid_search = GridSearchCV(svcpipe, parameters, cv=3, n_jobs=-1, verbose=3)
grid_search.fit(data, target)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 486 out of 486 | elapsed:  2.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('clean',
                                        FunctionTransformer(accept_sparse=False,
                                                            check_inverse=True,
                                                            func=<function clean_text at 0x7fbad9fee510>,
                                                            inv_kw_args=None,
                                                            inverse_func=None,
                                                            kw_args=None,
                                                            validate=False)),
                                       ('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                        

In [67]:
grid_search.best_params_

{'clf__C': 1,
 'clf__kernel': 'rbf',
 'vect__max_df': 0.7,
 'vect__max_features': 600,
 'vect__min_df': 0.01}

In [68]:
grid_search.best_score_

0.7516170081731536

In [69]:
sub = pd.DataFrame(model.predict(test.description),index=test.id, columns =['ratingCategory'])
sub.to_csv('2labelsvc.csv')


In [70]:
# create pipeline

pipe = Pipeline([('clean', clean),
                 ('vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                 ('clf', RandomForestClassifier())
                ])

parameters = {
    'vect__max_df': ( 0.05, 0.7, 0.75, 0.8),
    'vect__min_df': (.01, .02, .03),
    'vect__max_features': (300,500,800),
    'clf__n_estimators':(8, 10,15),
    'clf__max_depth':([15])
}



In [71]:
data = df.description
target = df.ratingCategory
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data, target)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  1.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('clean',
                                        FunctionTransformer(accept_sparse=False,
                                                            check_inverse=True,
                                                            func=<function clean_text at 0x7fbad9fee510>,
                                                            inv_kw_args=None,
                                                            inverse_func=None,
                                                            kw_args=None,
                                                            validate=False)),
                                       ('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                        

In [72]:
grid_search.best_params_

{'clf__max_depth': 15,
 'clf__n_estimators': 10,
 'vect__max_df': 0.7,
 'vect__max_features': 500,
 'vect__min_df': 0.01}

In [73]:
grid_search.best_score_

0.7371929174005748

In [10]:
model = grid_search.best_estimator_

In [11]:
test  = pd.read_csv('projects/DS-Unit-4-Sprint-1-NLP/module2-vector-representations/test.csv')

In [89]:
test.head()

,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [74]:
sub = pd.DataFrame(model.predict(test.description),index=test.id, columns =['ratingCategory'])
sub.to_csv('3nd_rfc.csv')

id,ratingCategory
3461,1
2604,1
3341,1
3764,1
2306,1
1438,1
2571,1
4148,1
4452,1


In [14]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=20)

In [15]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
# LSI
lsi = Pipeline([('clean', clean),
                 ('vect', vect),
                 ('svd', svd)
                ])

# Pipe
rfc = RandomForestClassifier()
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [16]:

parameters = {'clf__max_depth': (5,15,80),
     'clf__n_estimators': (50,100),
     'lsi__vect__max_df': (0.7,0.75,0.8,1),
     'lsi__vect__max_features': [100,500,1000],
     'lsi__vect__min_df': [0.05, 0.01],
    'lsi__svd__n_components': [50,100]
             }

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data, target)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1202 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  5.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('clean',
                                                         FunctionTransformer(accept_sparse=False,
                                                                             check_inverse=True,
                                                                             func=<function clean_text at 0x7fbb4802aa60>,
                                                                             inv_kw_args=None,
                                                                             inverse_func=None,
                                                                             kw_args=None,
                                                                             validate=False)),
                                                 

In [17]:
grid_search.best_score_


0.727430548281775

In [18]:
model2 = grid_search.best_estimator_

In [19]:
grid_search.best_params_

{'clf__max_depth': 80,
 'clf__n_estimators': 50,
 'lsi__svd__n_components': 50,
 'lsi__vect__max_df': 0.8,
 'lsi__vect__max_features': 1000,
 'lsi__vect__min_df': 0.01}

In [20]:
lsisub = pd.DataFrame(model2.predict(test.description),index=test.id, columns =['ratingCategory'])

In [75]:
sub.to_csv('lsi_rfc.csv')

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

1. 
  a. sentiment analysis is about trying to identify the emotional dimensions of natural language
  b. document classification is different because in many cases the classes are more objective, more related to literal meaning and structure. Emotional content can be diffucult to parse and relies more on context and subtle nuance. For example the same word,  "Great!" can be taken literaly to mean i am ahppy aobut something. or in a different context 'Great.' can mean i am nonplussed about a situation, taken sarcastically. you would need context to detect this, and in speech flesh humans rely on tone, body languge to convey extra information.
  c. we can create labeled sentiment data by letting humans evaluate the text and label it. they are a subjective approximation. a lot of scientific methodology would need to be used to get good data
  d. common applications would be to gain customer feedback, on a product or service. to identify problems. to classify users in a community 
2. Neural nets seem offer a 'shortcut' to impressive performance on classification problems, since theire complexity can extract structure without the need for multiple human assisted steps. however in this regard they are somewhat of a black box, in that the exact causual process to arrive at a prediction is not easily translatable to human thinking. 
